## CLEANUP AND DISTANCE MAPPING
* Code will get rid of any listing records with 0 reviews
* code will calculate distance of listing from Melbourne CBD
* code will calculate distance of listing from train station
* code will calculate distance of listing from CBD
* code will merge the station distance with the listing data

In [1]:
# import libraries
import math
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', None)

In [2]:


def get_distance(latA,longA,latB,longB)  :
    # radius of the Earth
    R = 6373.0

    #coordinates
    lat1 = math.radians(latA)
    lon1 = math.radians(longA)
    lat2 = math.radians(latB)
    lon2 = math.radians(longB)

    # change in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    #  Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2


    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return(distance)

In [3]:
# import listings 2018 csv to pd dataframe
path="../02-Data_Files_Directory/01-Raw_Data/listings_2018_prelimClean_JS.csv"
airbnb_df = pd.read_csv(path, low_memory=False)

In [4]:
airbnb_df.head()


,id,scrape_id,last_scraped,name,summary,neighborhood_overview,transit,access,interaction,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,9835,2.020000e+13,7/12/18,Beautiful Room & House,NaN,Very safe! Family oriented. Older age group.,"YES ! The bus (305,309) is exactly two blocks ...","Kitchen, backyard, upstairs lounge. We'd like ...",As much as the guest wants.,33057,Manju,21/8/09,AU,a few days or more,0%,NaN,f,NaN,1.0,1.0,"Bulleen, VIC, Australia",Balwyn North,Manningham,NaN,Bulleen,VIC,3105,Melbourne,"Bulleen, Australia",AU,Australia,-37.772684,145.092133,f,House,Private room,2,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,""Pets live on this property"",Do...",NaN,$60.00,NaN,NaN,NaN,NaN,1,$22.00,1,365,17 months ago,t,30,60,90,365,4,24/5/11,12/9/15,90.0,9.0,10.0,10.0,10.0,9.0,9.0,0.04
1,10803,2.020000e+13,7/12/18,Room in Cool Deco Apartment in Brunswick,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,Easy transport options - the tram is right out...,Wifi. Bathroom and kitchen is shared but I mos...,I'm home mostly but sometimes I may be working...,38901,Lindsay,16/9/09,"Melbourne, Victoria, Australia",NaN,NaN,NaN,f,Brunswick,1.0,1.0,"Brunswick East, VIC, Australia",Brunswick,Moreland,NaN,Brunswick East,VIC,3057,Melbourne,"Brunswick East, Australia",AU,Australia,-37.766505,144.980736,t,Apartment,Private room,2,1.0,1.0,1.0,Futon,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,H...",226.0,$35.00,$200.00,$803.00,$0.00,$9.00,1,$15.00,3,45,6 weeks ago,t,4,9,33,213,108,12/1/13,26/11/18,87.0,9.0,8.0,9.0,9.0,9.0,9.0,1.50
2,12936,2.020000e+13,7/12/18,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,Guests have exclusive and private access to th...,We don't always get to meet all our guests in ...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",within an hour,100%,NaN,t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",St Kilda,Port Phillip,NaN,St Kilda,VIC,3182,Melbourne,"St Kilda, Australia",AU,Australia,-37.859755,144.977369,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15
3,15246,2.020000e+13,7/12/18,Large private room-close to city,"Comfortable, relaxed house, a home away from ...","This is a great neighbourhood â€“ it is quiet,...",Public transport is super convenient with a ch...,You are welcome to make yourself at home in th...,Guest's privacy will be respected at all times...,59786,Eleni,3/12/09,"Melbourne, Victoria, Australia",NaN,NaN,NaN,f,Thornbury,3.0,3.0,"Thornbury, VIC, Australia",Thornbury,Darebin,NaN,Thornbury,VIC,3071,Melbourne,"Thornbury, Australia",AU,Australia,-37.758971,144.989228,t,House,Private room,1,1.5,1.0,1.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Free parking on pre...",0.0,$50.00,$250.00,$920.00,$200.00,$15.00,1,$20.00,2,200,11 months ago,t,0,0,0,0,29,31/12/10,15/5/17,94.0,9.0,9.0,

In [5]:
airbnb_df2 = (airbnb_df.loc[airbnb_df['room_type'] == 'Entire home/apt']
             .loc[airbnb_df['number_of_reviews'] > 0])
# feel free to comment  fields we do not need or uncomment  fields we do
airbnb_df2 = airbnb_df2[
            ['id',
            #'scrape_id', #i dont see a significance of scrape ID
            #'last_scraped', #shows only 2 dates, dec 7 and 8 most likely when the data was last updated
            'name',
            #'summary',
            'neighborhood_overview',
            'transit',
            #'access',
            #'interaction',
            'host_id',
            'host_name',
            'host_since',
            'host_location',
            #'host_response_time',
            #'host_response_rate',
            #'host_acceptance_rate',
            'host_is_superhost',
            'host_neighbourhood', 
            'host_listings_count',
            'host_total_listings_count',
            'street',
            #'neighbourhood', # neighbourhood_cleansed shows a more accurate location
            'neighbourhood_cleansed',
            #'neighbourhood_group_cleansed',
            'city',
            'state',
            'zipcode',
            #'market', #we know everything should be in melbourne plus there are confusing markets
            #'smart_location', #ucomment if needed
            #'country_code', #we know everything is in AU
            #'country', #we know everything is in AU
            'latitude',
            'longitude',
            'is_location_exact',
            'property_type',
            #'room_type', # no need fo this field since we are only using 'Entire home/apt'
            'accommodates',
            'bathrooms',
            'bedrooms',
            'beds',
            'bed_type',
            'amenities',
            'square_feet',
            'price',
            'weekly_price',
            'monthly_price',
            'security_deposit',
            'cleaning_fee',
            'guests_included',
            'extra_people',
            'minimum_nights',
            'maximum_nights',
            'calendar_updated',
            'has_availability',
            'availability_30',
            'availability_60',
            'availability_90',
            'availability_365',
            'number_of_reviews',
            'first_review',
            'last_review',
            'review_scores_rating',
            'review_scores_accuracy',
            'review_scores_cleanliness',
            'review_scores_checkin',
            'review_scores_communication',
            'review_scores_location',
            'review_scores_value',
            'reviews_per_month']]

airbnb_df2['distance_from_cbd'] = ""
airbnb_df2

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,
5,38271,Melbourne - Old Trafford Apartment,Our street is quiet & secluded but within walk...,If you're arriving via the airport (Tullamarin...,164193,Daryl,12/7/10,"Berwick, Victoria, Australia",t,NaN,1.0,1.0,"Berwick, VIC, Australia",Casey,Berwick,VIC,3806,-38.057254,145.339356,t,Apartment,5,1.0,3.0,3.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",NaN,$98.00,$540.00,NaN,$0.00,$0.00,1,$30.00,1,14,2 weeks ago,t,12,27,57,328,112,24/11/10,1/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.14,
8,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,Oakleigh is one of the most convenient and div...,"Monash University, Clayton Campus is a 10 minu...",189684,Allan,3/8/10,"Melbourne, Victoria, Australia",t,NaN,2.0,2.0,"Oakleigh East, VIC, Australia",Monash,Oakleigh East,VIC,3166,-37.899601,145.114475,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",388.0,$98.00,$535.00,NaN,$500.00,$40.00,1,$0.00,2,365,today,t,7,25,35,310,168,5/12/10,3/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.72,
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22545,30412315,Doncaster Modern Oasis. Netflix / Pool / Break...,Doncaster is perfectly placed to explore many ...,With a large bus station at Westfield Shopping...,24980853,Paul,18/12/14,"Melbourne, Victoria, Australia",f,NaN,2.0,2.0,"Doncaster, VIC, Australia",Manningham,Doncaster,VIC,3108,-37.782288,145.127

In [6]:
# add distance from city in KMs
cbdLat =-37.814326
cbdLong=144.963149

for index, row in airbnb_df2.iterrows():
    distance = get_distance(row['latitude'], #latA
                            row['longitude'], #longA
                        cbdLat, #latB
                        cbdLong) #longB
    airbnb_df2.loc[index, 'distance_from_cbd'] = distance
airbnb_df2

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,5.20517
5,38271,Melbourne - Old Trafford Apartment,Our street is quiet & secluded but within walk...,If you're arriving via the airport (Tullamarin...,164193,Daryl,12/7/10,"Berwick, Victoria, Australia",t,NaN,1.0,1.0,"Berwick, VIC, Australia",Casey,Berwick,VIC,3806,-38.057254,145.339356,t,Apartment,5,1.0,3.0,3.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",NaN,$98.00,$540.00,NaN,$0.00,$0.00,1,$30.00,1,14,2 weeks ago,t,12,27,57,328,112,24/11/10,1/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.14,42.6539
8,43429,Tranquil Javanese-Style Apartment in Oakleigh ...,Oakleigh is one of the most convenient and div...,"Monash University, Clayton Campus is a 10 minu...",189684,Allan,3/8/10,"Melbourne, Victoria, Australia",t,NaN,2.0,2.0,"Oakleigh East, VIC, Australia",Monash,Oakleigh East,VIC,3166,-37.899601,145.114475,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",388.0,$98.00,$535.00,NaN,$500.00,$40.00,1,$0.00,2,365,today,t,7,25,35,310,168,5/12/10,3/12/18,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.72,16.3273
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,5.437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22545,30412315,Doncaster Modern Oasis. Netflix / Pool / Break...,Doncaster is perfectly placed to explore many ...,With a large bus station at Westfield Shopping...,24980853,Paul,18/12/14,"Melbourne, Victoria, Australia",f,NaN,2.0,2.0,"Doncaster, VIC, Australia",Manningham,Donc

In [7]:
max_distance = 12

airbnb_df3 = airbnb_df2.loc[airbnb_df2['distance_from_cbd'] <= max_distance]

# save data to CSV *****LINE DEPRICATED
#airbnb_df3.to_csv('../02-Data_Files_Directory/02-Clean_Data/listings_2018_clean2_RS.csv')
airbnb_df3

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$159.00,"$1,253.00","$4,452.00",$450.00,$100.00,2,$28.00,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94.0,9.0,10.0,9.0,9.0,9.0,9.0,0.15,5.20517
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,$98.00,$800.00,NaN,$200.00,$80.00,1,$20.00,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98.0,10.0,10.0,10.0,10.0,10.0,10.0,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,$190.00,"$1,743.00","$5,572.00",$500.00,$110.00,1,$0.00,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97.0,10.0,10.0,10.0,10.0,9.0,10.0,0.60,5.437
12,51592,Central City Warehouse Apartment,The apartment is located in the centre of Melb...,"Yes, minutes to trains, trams and buses but mo...",190879,Michelle,4/8/10,"Victoria, Australia",t,Central Business District,1.0,1.0,"Melbourne, VIC, Australia",Melbourne,Melbourne,VIC,3000,-37.814856,144.962672,t,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,$228.00,"$1,575.00","$5,100.00",$250.00,$40.00,2,$40.00,2,90,3 weeks ago,t,13,39,56,331,196,22/2/11,1/12/18,99.0,10.0,9.0,10.0,10.0,10.0,10.0,2.07,0.072335
16,66754,Richmond Super Central 60s Cool 1BR APT+WIFI+AC,A stay at our hip apartment will mean you can ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17.0,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.820916,144.993839,t,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,$138.00,"$1,183.00","$4,172.00",$450.00,$100.00,2,$28.00,3,28,today,t,11,19,49,324,43,8/3/11,11/11/18,95.0,9.0,10.0,10.0,10.0,10.0,9.0,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,Melbourne CBD 2 Bedrooms/å¢¨å°”æœ¬å¸‚ä¸­å¿ƒ2æˆ...,NaN,NaN,127327853,Nathan,24/4/17,"Melbourne, Victoria, Australia",f,Central Business District,1.0,1.0,"Melbourne, VIC, Australia",Melbourne,Melbourne,VIC,3000,-37.80871

In [8]:
# #load station data DEPRICATED
# path2="clean_data_rs/ptv_metro_train_station.csv"
# stations_df = pd.read_csv(path2)
# stations_df

### Jason's clean up code starts here

In [9]:
#Check data info for null fields - Total records are 9890
airbnb_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9890 entries, 2 to 22698
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           9890 non-null   int64  
 1   name                         9890 non-null   object 
 2   neighborhood_overview        7230 non-null   object 
 3   transit                      7523 non-null   object 
 4   host_id                      9890 non-null   int64  
 5   host_name                    9889 non-null   object 
 6   host_since                   9889 non-null   object 
 7   host_location                9885 non-null   object 
 8   host_is_superhost            9889 non-null   object 
 9   host_neighbourhood           8514 non-null   object 
 10  host_listings_count          9889 non-null   float64
 11  host_total_listings_count    9889 non-null   float64
 12  street                       9890 non-null   object 
 13  neighbourhood_cle

In [10]:
#Replace null (missing) value with relevant data


airbnb_df4 = airbnb_df3.copy()

    #Fill NaN values with value "missing"
airbnb_df4['host_name'] = airbnb_df4['host_name'].fillna("Missing")
airbnb_df4['city'] = airbnb_df4['city'].fillna("Missing")

    #Fill NaN values with value "f"
airbnb_df4['host_is_superhost'] = airbnb_df4['host_is_superhost'].fillna("f")
    
    #Fill NaN values with value "0"

airbnb_df4['host_total_listings_count'] = airbnb_df4['host_total_listings_count'].fillna("0")

airbnb_df4['bathrooms'] = airbnb_df4['bathrooms'].fillna("0")

airbnb_df4['bedrooms'] = airbnb_df4['bedrooms'].fillna("0")

airbnb_df4['beds'] = airbnb_df4['beds'].fillna("0")

airbnb_df4['price'] = airbnb_df4['price'].fillna("0")

airbnb_df4['weekly_price'] = airbnb_df4['weekly_price'].fillna("0")

airbnb_df4['monthly_price'] = airbnb_df4['monthly_price'].fillna("0")

airbnb_df4['security_deposit'] = airbnb_df4['security_deposit'].fillna("0")

airbnb_df4['cleaning_fee'] = airbnb_df4['cleaning_fee'].fillna("0")

airbnb_df4['guests_included'] = airbnb_df4['guests_included'].fillna("0")

airbnb_df4['extra_people'] = airbnb_df4['extra_people'].fillna("0")

airbnb_df4['number_of_reviews'] = airbnb_df4['number_of_reviews'].fillna("0")

airbnb_df4['review_scores_rating'] = airbnb_df4['review_scores_rating'].fillna("0")

airbnb_df4['review_scores_accuracy'] = airbnb_df4['review_scores_accuracy'].fillna("0")

airbnb_df4['review_scores_cleanliness'] = airbnb_df4['review_scores_cleanliness'].fillna("0")

airbnb_df4['review_scores_checkin'] = airbnb_df4['review_scores_checkin'].fillna("0")

airbnb_df4['review_scores_communication'] = airbnb_df4['review_scores_communication'].fillna("0")

airbnb_df4['review_scores_location'] = airbnb_df4['review_scores_location'].fillna("0")

airbnb_df4['review_scores_value'] = airbnb_df4['review_scores_value'].fillna("0")

airbnb_df4['reviews_per_month'] = airbnb_df4['reviews_per_month'].fillna("0")


    #Fill NaN values with value "Victoria"
airbnb_df4['state'] = airbnb_df4['state'].fillna("Vic")

    #Fill NaN values with value "0000" - CONVERT ALL TO STRING? - dtype('O') - a type of string so I think ok.

airbnb_df4['zipcode'] = airbnb_df4['zipcode'].fillna("0000")
    

In [11]:
# - the followng columns have incorrect dtypes

airbnb_df5 = airbnb_df4.copy()

    #Should be floats (decimals)
    #df['DataFrame Column'] = df['DataFrame Column'].astype(float)
airbnb_df5['bathrooms'] = airbnb_df5['bathrooms'].astype(float)

airbnb_df5['bedrooms'] = airbnb_df5['bedrooms'].astype(float)

airbnb_df5['beds'] = airbnb_df5['beds'].astype(float)

airbnb_df5['reviews_per_month'] = airbnb_df5['reviews_per_month'].astype(float)


In [12]:
#Converting currency string to float (decimal)
#https://pbpython.com/pandas_dtypes.html

#Function to strip dollar signs and comma and change to float - don't quite understand but works!!
def convert_currency(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

In [13]:
airbnb_df5['price'] = airbnb_df5['price'].apply(convert_currency)
airbnb_df5['weekly_price'] = airbnb_df5['weekly_price'].apply(convert_currency)
airbnb_df5['monthly_price'] = airbnb_df5['monthly_price'].apply(convert_currency)
airbnb_df5['security_deposit'] = airbnb_df5['security_deposit'].apply(convert_currency)
airbnb_df5['cleaning_fee'] = airbnb_df5['cleaning_fee'].apply(convert_currency)
airbnb_df5['extra_people'] = airbnb_df5['extra_people'].apply(convert_currency)

In [14]:
 #Should be integer (whole number)

airbnb_df5['host_total_listings_count'] = airbnb_df5['host_total_listings_count'].astype(int)
airbnb_df5['review_scores_rating'] = airbnb_df5['review_scores_rating'].astype(int)
airbnb_df5['review_scores_accuracy'] = airbnb_df5['review_scores_accuracy'].astype(int)
airbnb_df5['review_scores_cleanliness'] = airbnb_df5['review_scores_cleanliness'].astype(int)
airbnb_df5['review_scores_checkin'] = airbnb_df5['review_scores_rating'].astype(int)
airbnb_df5['review_scores_communication'] = airbnb_df5['review_scores_communication'].astype(int)
airbnb_df5['review_scores_location'] = airbnb_df5['review_scores_location'].astype(int)
airbnb_df5['review_scores_value'] = airbnb_df5['review_scores_value'].astype(int)

#check that data types have been corrected
airbnb_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9890 entries, 2 to 22698
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           9890 non-null   int64  
 1   name                         9890 non-null   object 
 2   neighborhood_overview        7230 non-null   object 
 3   transit                      7523 non-null   object 
 4   host_id                      9890 non-null   int64  
 5   host_name                    9890 non-null   object 
 6   host_since                   9889 non-null   object 
 7   host_location                9885 non-null   object 
 8   host_is_superhost            9890 non-null   object 
 9   host_neighbourhood           8514 non-null   object 
 10  host_listings_count          9889 non-null   float64
 11  host_total_listings_count    9890 non-null   int32  
 12  street                       9890 non-null   object 
 13  neighbourhood_cle

In [15]:
airbnb_df5.head()

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94,9,10,94,9,9,9,0.15,5.20517
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98,10,10,98,10,10,10,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,VIC,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97,10,10,97,10,9,10,0.60,5.437
12,51592,Central City Warehouse Apartment,The apartment is located in the centre of Melb...,"Yes, minutes to trains, trams and buses but mo...",190879,Michelle,4/8/10,"Victoria, Australia",t,Central Business District,1.0,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,VIC,3000,-37.814856,144.962672,t,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,3 weeks ago,t,13,39,56,331,196,22/2/11,1/12/18,99,10,9,99,10,10,10,2.07,0.072335
16,66754,Richmond Super Central 60s Cool 1BR APT+WIFI+AC,A stay at our hip apartment will mean you can ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"Richmond, VIC, Australia",Yarra,Richmond,VIC,3121,-37.820916,144.993839,t,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,today,t,11,19,49,324,43,8/3/11,11/11/18,95,9,10,95,10,10,9,0.46,2.79455


In [16]:
#strip leading and trailing space
airbnb_df5['city'] = airbnb_df5['city'].str.strip()

In [17]:
#See what spelling mistalkes need fixing in city column
airbnb_df5City = airbnb_df5["city"].unique()
airbnb_df5City.sort()
print(airbnb_df5City)
len(airbnb_df5City)

['Abbotsford' 'Abbotsford, Melbourne' 'Airport West' 'Albert Park'
 'Albert Park. Melbourne' 'Alphington' 'Altona North' 'Armadale'
 'Ascot Vale' 'Ashburton' 'BRUNSWICK EAST' 'Balaclava' 'Balwyn'
 'Balwyn North' 'Bellfield' 'Braybrook' 'Brighton' 'Brighton East'
 'Brooklyn' 'Brunswick' 'Brunswick / Melbourne' 'Brunswick East'
 'Brunswick East, Melbourne' 'Brunswick West' 'Brunswick West, Melbourne'
 'Brunswick west' 'Burnley' 'Camberwell' 'Canterbury' 'Carlton'
 'Carlton North' 'Carlton, Melbourne' 'Carnegie' 'Caulfield'
 'Caulfield East' 'Caulfield North' 'Caulfield South'
 'City of Port Phillip' 'Clifton Hill' 'Coburg' 'Coburg (Melbourne)'
 'Coburg North' 'Collingwood' 'Collingwood, Melbourne' 'Cremorne'
 'Dockland' 'Docklands' 'Eaglemont' 'East Melbourne' 'East St. Kilda'
 'East st kilda' 'Elsternwick' 'Elwood' 'Essendon' 'Essendon North'
 'Essendon West' 'Essendon, Victoria, AU' 'FITZROY' 'Fairfield' 'Fawkner'
 'Fitzroy' 'Fitzroy North' 'Fitzroy, Melbourne' 'Flemington' 'Footscray'

167

In [18]:
#Fix spelling mistake in city column
airbnb_df5["city"] = airbnb_df5["city"].replace({"Abbotsford, Melbourne":"Abbotsford","Albert Park. Melbourne":"Albert Park","BRUNSWICK EAST":"Brunswick East","Brunswick / Melbourne":"Brunswick","Brunswick East, Melbourne":"Brunswick East","Brunswick West, Melbourne":"Brunswick West","Brunswick west":"Brunswick West","Carlton, Melbourne":"Carlton","Coburg (Melbourne)":"Coburg","Collingwood, Melbourne":"Collingwood","Dockland":"Docklands","East St. Kilda":"St Kilda East","East st kilda":"St Kilda East","Essendon, Victoria, AU":"Essendon","FITZROY":"Fitzroy","Fitzroy, Melbourne":"Fitzroy","Glen Iris, Victoria, AU":"Glen Iris","Hawthorn east":"Hawthorn East","Hawthorn, Melbourne":"Hawthorn","Ivanhoe (Melbourne)":"Ivanhoe","Melborne":"Melbourne","Melbourne City":"Melbourne","Melbourne Victoria":"Melbourne","Melbourne, Victoria, AU":"Melbourne","Middle Park Melbourne":"Middle Park","North Melbourne, Victoria, AU":"North Melbourne","Northcote, Victoria, AU":"Northcote","Praharan":"Prahran","Richmond, Victoria, AU":"Richmond","Saint Kilda, Victoria, AU":"St Kilda East","South Yarra, Victoria, AU":"South Yarra","Southbank Melbourne":"Southbank","Southbank, Melbourne":"Southbank","Southbank, Victoria, AU":"Southbank","Saint Kilda East":"St Kilda East","Saint Kilda":"St Kilda","Saint Kilda West":"St Kilda West","St Kilda West Melbourne":"St Kilda West","St kilda":"St Kilda","St. Kilda":"St Kilda","St. Kilda West":"St Kilda West","StKilda East":"St Kilda East","Surrey Hills, Victoria, AU":"Surrey Hills","Thornbury, Victoria, AU":"Thornbury","Toorak, Melbourne.":"Toorak","VIC":"Melbourne","Victoria":"Melbourne","brunswick":"Brunswick","melbourne":"Melbourne","st kilda":"St Kilda"})
airbnb_df5["state"] = "Vic"

In [19]:
#See unique values again
airbnb_df5City2 = airbnb_df5["city"].unique()
airbnb_df5City2.sort()
print(airbnb_df5City2)
len(airbnb_df5City2)

['Abbotsford' 'Airport West' 'Albert Park' 'Alphington' 'Altona North'
 'Armadale' 'Ascot Vale' 'Ashburton' 'Balaclava' 'Balwyn' 'Balwyn North'
 'Bellfield' 'Braybrook' 'Brighton' 'Brighton East' 'Brooklyn' 'Brunswick'
 'Brunswick East' 'Brunswick West' 'Burnley' 'Camberwell' 'Canterbury'
 'Carlton' 'Carlton North' 'Carnegie' 'Caulfield' 'Caulfield East'
 'Caulfield North' 'Caulfield South' 'City of Port Phillip' 'Clifton Hill'
 'Coburg' 'Coburg North' 'Collingwood' 'Cremorne' 'Docklands' 'Eaglemont'
 'East Melbourne' 'Elsternwick' 'Elwood' 'Essendon' 'Essendon North'
 'Essendon West' 'Fairfield' 'Fawkner' 'Fitzroy' 'Fitzroy North'
 'Flemington' 'Footscray' 'Gardenvale' 'Glen Huntly' 'Glen Iris'
 'Hadfield' 'Hawthorn' 'Hawthorn East' 'Heidelberg' 'Heidelberg Heights'
 'Ivanhoe' 'Ivanhoe East' 'Keilor East' 'Kensington' 'Kew' 'Kew East'
 'Kingsville' 'Kooyong' 'Maidstone' 'Malvern' 'Malvern East' 'Maribyrnong'
 'Melbourne' 'Middle Park' 'Missing' 'Moonee Ponds' 'Moreland City'
 'Newport

117

In [20]:
airbnb_df5

,id,name,neighborhood_overview,transit,host_id,host_name,host_since,host_location,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,is_location_exact,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,St Kilda 1BR APT+BEACHSIDE+VIEWS+PARKING+WIFI+AC,A stay at our apartment means you can enjoy so...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,t,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,today,t,10,26,51,326,15,4/8/10,7/11/18,94,9,10,94,9,9,9,0.15,5.20517
10,47100,TrÃ¨s Charming in Fabulous Richmond,Bridge Road (once famous for discount shopping...,There are two tram lines right outside the apa...,212071,Loren,24/8/10,"Melbourne, Victoria, Australia",t,Richmond,4.0,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,t,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",NaN,98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,2 weeks ago,t,13,13,18,22,176,9/9/10,1/11/18,98,10,10,98,10,10,10,1.75,3.64957
11,50492,St Kilda Central LUXE 2BR APT+TWO COURTYARDS+WIFI,St Kilda is one of Melbourneâ€™s most vibrant ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,t,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",0.0,190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,today,t,10,22,44,316,44,30/11/12,2/12/18,97,10,10,97,10,9,10,0.60,5.437
12,51592,Central City Warehouse Apartment,The apartment is located in the centre of Melb...,"Yes, minutes to trains, trams and buses but mo...",190879,Michelle,4/8/10,"Victoria, Australia",t,Central Business District,1.0,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,t,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",NaN,228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,3 weeks ago,t,13,39,56,331,196,22/2/11,1/12/18,99,10,9,99,10,10,10,2.07,0.072335
16,66754,Richmond Super Central 60s Cool 1BR APT+WIFI+AC,A stay at our hip apartment will mean you can ...,Our apartment is located within walking distan...,50121,Frank & Vince,31/10/09,"Melbourne, Victoria, Australia",t,St Kilda,17.0,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,t,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",0.0,138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,today,t,11,19,49,324,43,8/3/11,11/11/18,95,9,10,95,10,10,9,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,Melbourne CBD 2 Bedrooms/å¢¨å°”æœ¬å¸‚ä¸­å¿ƒ2æˆ...,NaN,NaN,127327853,Nathan,24/4/17,"Melbourne, Victoria, Australia",f,Central Business District,1.0,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,t,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",NaN,156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,2 days ago,t,15,32,52

In [21]:
#Drop irrlevant colum
#'scrape_id', #i dont see a significance of scrape ID
            #'last_scraped', #shows only 2 dates, dec 7 and 8 most likely when the data was last updated
        
            #'summary',
   
            #'access',
            #'interaction',
           
            #'host_response_time',
            #'host_response_rate',
            #'host_acceptance_rate',
          
            #'neighbourhood', # neighbourhood_cleansed shows a more accurate location
        
            #'neighbourhood_group_cleansed',
         
            #'market', #we know everything should be in melbourne plus there are confusing markets
            #'smart_location', #ucomment if needed
            #'country_code', #we know everything is in AU
            #'country', #we know everything is in AU
    
            #'room_type', # no need fo this field since we are only using 'Entire home/apt'



airbnb_df6 = airbnb_df5.drop(["name","neighborhood_overview","transit",
                              "host_since", "host_location",
                              "host_listings_count", "host_neighbourhood",
                              "is_location_exact", "calendar_updated",
                              "first_review", "last_review", "square_feet"], axis = 1)
airbnb_df6

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.20517
10,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.64957
11,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437
12,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335
16,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,127327853,Nathan,f,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,t,15,32,52,52,2,100,10,10,100,10,10,10,2.00,0.700146
22645,30468406,30322920,Robyn,f,1,"Brunswick West, VIC, Australia",Moreland,Brunswick West,Vic,3055,-37.762380,144.949909,House,6,1.5,3.0,6.0,Real Bed,"{Wifi,Kitchen,""Free parking on premises"",Washe...",199.0,0.0,0.0,0.0,60.0,4,20.0,2,1125,t,15,34,46,97,1,0,0,0,0,0,0,0,1.00,5.89398
22653,30470482,36410227,Sabrina,f,276,"Port Melbourne, Australia",Port Phillip,Port Melbourne,Vic,3207,-37.844490,144.943306,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",140.0,0.0,0.0,250.0,140.0,4,0.0,3,1125,t,2,25,55,146,1,0,0,0,0,0,0,0,1.00,3.781
22676,30483362,154023219,Aishah,f,1,"Preston, VIC, Australia",Darebin,Preston,Vic,3072,-37.742043,145.002332,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",71.0,0.0,0.0,0.0,0.0,1,0.0,1,7,t,21,40,40,40,1,0,0,0,0,0,0,0,1.00,8.74685


In [22]:
airbnb_df6.to_csv("../02-Data_Files_Directory/02-Clean_Data/airbnbFINAL_JS.csv", index =False) 

### Swobabika's clean up starts here.

In [23]:
# Get all unique property types.
unique_property = airbnb_df6['property_type'].value_counts()
print(unique_property)

Apartment             7393
House                 1132
Townhouse              377
Condominium            323
Serviced apartment     281
Loft                   125
Guesthouse              93
Guest suite             70
Bungalow                33
Villa                   21
Cottage                 12
Tiny house               8
Boat                     6
Other                    5
Barn                     3
Bed and breakfast        2
Cabin                    1
Nature lodge             1
Camper/RV                1
Treehouse                1
Resort                   1
Castle                   1
Name: property_type, dtype: int64


In [24]:
# Filter property_type to keep those with more than 100 datapoints.
airbnb_df7 = airbnb_df6.loc[(airbnb_df6["property_type"]=="Apartment") | 
                                             (airbnb_df6["property_type"]=="House") |
                                            (airbnb_df6["property_type"]=="Townhouse") |
                                            (airbnb_df6["property_type"]=="HouseCondominium") |
                                            (airbnb_df6["property_type"]=="Serviced apartment") |
                                            (airbnb_df6["property_type"]=="Loft") ,:]
                                            
airbnb_df7

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.20517
10,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.64957
11,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437
12,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335
16,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,127327853,Nathan,f,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,t,15,32,52,52,2,100,10,10,100,10,10,10,2.00,0.700146
22645,30468406,30322920,Robyn,f,1,"Brunswick West, VIC, Australia",Moreland,Brunswick West,Vic,3055,-37.762380,144.949909,House,6,1.5,3.0,6.0,Real Bed,"{Wifi,Kitchen,""Free parking on premises"",Washe...",199.0,0.0,0.0,0.0,60.0,4,20.0,2,1125,t,15,34,46,97,1,0,0,0,0,0,0,0,1.00,5.89398
22653,30470482,36410227,Sabrina,f,276,"Port Melbourne, Australia",Port Phillip,Port Melbourne,Vic,3207,-37.844490,144.943306,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",140.0,0.0,0.0,250.0,140.0,4,0.0,3,1125,t,2,25,55,146,1,0,0,0,0,0,0,0,1.00,3.781
22676,30483362,154023219,Aishah,f,1,"Preston, VIC, Australia",Darebin,Preston,Vic,3072,-37.742043,145.002332,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",71.0,0.0,0.0,0.0,0.0,1,0.0,1,7,t,21,40,40,40,1,0,0,0,0,0,0,0,1.00,8.74685


In [25]:
# Making sure irrelevant data is removed.
unique_property = airbnb_df7['property_type'].value_counts()
print(unique_property)

Apartment             7393
House                 1132
Townhouse              377
Serviced apartment     281
Loft                   125
Name: property_type, dtype: int64


### Swobabika added additional clean up here.

In [26]:
# View value counts of suburbs with least datapoints
airbnb_df7['city'].value_counts().tail(60)

Ascot Vale              21
West Footscray          19
Moonee Ponds            18
Malvern                 16
Carnegie                16
Seddon                  16
Fairfield               15
Newport                 14
Ivanhoe                 14
Travancore              14
Essendon                13
Maidstone               13
Caulfield               12
Balwyn                  12
Ripponlea               11
Reservoir               11
Glen Huntly              9
Alphington               8
Burnley                  8
Brighton East            7
Balwyn North             7
Princes Hill             7
Heidelberg Heights       7
Altona North             7
South Kingsville         6
Braybrook                6
Kingsville               6
Surrey Hills             6
Canterbury               6
Missing                  5
Pascoe Vale South        5
Pascoe Vale              5
Coburg North             5
Keilor East              5
Sunshine                 5
Spotswood                4
Gardenvale               4
K

In [27]:
# Filter data for suburbs with more than 20 datapoints only.
airbnb_df8 = airbnb_df7.groupby('city').filter(lambda x : len(x)>20)
airbnb_df8

,id,host_id,host_name,host_is_superhost,host_total_listings_count,street,neighbourhood_cleansed,city,state,zipcode,latitude,longitude,property_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,distance_from_cbd
2,12936,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.859755,144.977369,Apartment,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",159.0,1253.0,4452.0,450.0,100.0,2,28.0,3,28,t,10,26,51,326,15,94,9,10,94,9,9,9,0.15,5.20517
10,47100,212071,Loren,t,4,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.818037,145.004416,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",98.0,800.0,0.0,200.0,80.0,1,20.0,4,30,t,13,13,18,22,176,98,10,10,98,10,10,10,1.75,3.64957
11,50492,50121,Frank & Vince,t,17,"St Kilda, VIC, Australia",Port Phillip,St Kilda,Vic,3182,-37.861275,144.980379,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Buzzer/wireless int...",190.0,1743.0,5572.0,500.0,110.0,1,0.0,3,28,t,10,22,44,316,44,97,10,10,97,10,9,10,0.60,5.437
12,51592,190879,Michelle,t,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.814856,144.962672,Loft,4,1.5,1.0,2.0,Real Bed,"{TV,Internet,Wifi,Kitchen,""Paid parking off pr...",228.0,1575.0,5100.0,250.0,40.0,2,40.0,2,90,t,13,39,56,331,196,99,10,9,99,10,10,10,2.07,0.072335
16,66754,50121,Frank & Vince,t,17,"Richmond, VIC, Australia",Yarra,Richmond,Vic,3121,-37.820916,144.993839,Apartment,4,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",138.0,1183.0,4172.0,450.0,100.0,2,28.0,3,28,t,11,19,49,324,43,95,9,10,95,10,10,9,0.46,2.79455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22537,30410904,127327853,Nathan,f,1,"Melbourne, VIC, Australia",Melbourne,Melbourne,Vic,3000,-37.808718,144.959530,Apartment,5,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Pool,Kitchen,Gym,E...",156.0,0.0,0.0,0.0,75.0,1,0.0,1,1125,t,15,32,52,52,2,100,10,10,100,10,10,10,2.00,0.700146
22645,30468406,30322920,Robyn,f,1,"Brunswick West, VIC, Australia",Moreland,Brunswick West,Vic,3055,-37.762380,144.949909,House,6,1.5,3.0,6.0,Real Bed,"{Wifi,Kitchen,""Free parking on premises"",Washe...",199.0,0.0,0.0,0.0,60.0,4,20.0,2,1125,t,15,34,46,97,1,0,0,0,0,0,0,0,1.00,5.89398
22653,30470482,36410227,Sabrina,f,276,"Port Melbourne, Australia",Port Phillip,Port Melbourne,Vic,3207,-37.844490,144.943306,Apartment,4,1.0,2.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Pool,Kitc...",140.0,0.0,0.0,250.0,140.0,4,0.0,3,1125,t,2,25,55,146,1,0,0,0,0,0,0,0,1.00,3.781
22676,30483362,154023219,Aishah,f,1,"Preston, VIC, Australia",Darebin,Preston,Vic,3072,-37.742043,145.002332,Apartment,2,1.0,1.0,1.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,""Smoking a...",71.0,0.0,0.0,0.0,0.0,1,0.0,1,7,t,21,40,40,40,1,0,0,0,0,0,0,0,1.00,8.74685


In [28]:
# Check suburb value counts after filtering.
airbnb_df8['city'].value_counts()

Melbourne          2732
Southbank           658
St Kilda            627
South Yarra         566
Docklands           346
Richmond            334
Carlton             221
Fitzroy             205
Collingwood         176
Elwood              172
South Melbourne     167
North Melbourne     163
Brunswick           154
Port Melbourne      151
West Melbourne      139
Prahran             134
Northcote           128
Abbotsford          124
St Kilda East       114
East Melbourne      110
Fitzroy North       109
Windsor             101
Brunswick East       82
Hawthorn             74
St Kilda West        64
Thornbury            55
Balaclava            55
Toorak               54
Carlton North        52
Brighton             49
Caulfield North      46
Footscray            46
Albert Park          43
Yarraville           42
Clifton Hill         40
Kensington           40
Preston              39
Coburg               37
Hawthorn East        36
Williamstown         36
Brunswick West       34
Elsternwick     

In [29]:
# remove train station data merging and move to 01_Calculate_Nearest_Station.ipynb
airbnb_df8.to_csv("../02-Data_Files_Directory/02-Clean_Data/airbnbFINAL_Swobabika.csv", index =False)